# Pair Trading of GLD and GDX

In [ ]:
import numpy as np

In [ ]:
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import statsmodels.api as sm

In [ ]:
df1=pd.read_excel('GLD.xls')

In [ ]:
df2=pd.read_excel('GDX.xls')

In [ ]:
df=pd.merge(df1, df2, on='Date', suffixes=('_GLD', '_GDX'))

In [ ]:
df.set_index('Date', inplace=True)

In [ ]:
df.sort_index(inplace=True)

In [ ]:
trainset=np.arange(0, 252)

In [ ]:
testset=np.arange(trainset.shape[0], df.shape[0])

## Determine hedge ratio on trainset

In [ ]:
model=sm.OLS(df.loc[:, 'Adj Close_GLD'].iloc[trainset], df.loc[:, 'Adj Close_GDX'].iloc[trainset])

In [ ]:
results=model.fit()

In [ ]:
hedgeRatio=results.params

In [ ]:
hedgeRatio

##  spread = GLD - hedgeRatio*GDX

In [ ]:
spread=df.loc[:, 'Adj Close_GLD']-heHSdgeRatio[0]*df.loc[:, 'Adj Close_GDX']

In [ ]:
plt.plot(spread.iloc[trainset])

In [ ]:
plt.plot(spread.iloc[testset])

In [ ]:
spreadMean=np.mean(spread.iloc[trainset])

In [ ]:
spreadMean

In [ ]:
spreadStd=np.std(spread.iloc[trainset])

In [ ]:
spreadStd

In [ ]:
df['zscore']=(spread-spreadMean)/spreadStd

In [ ]:
df['positions_GLD_Long']=0

In [ ]:
df['positions_GDX_Long']=0

In [ ]:
df['positions_GLD_Short']=0

In [ ]:
df['positions_GDX_Short']=0

In [ ]:
df.loc[df.zscore>=2, ('positions_GLD_Short', 'positions_GDX_Short')]=[-1, 1] # Short spread

In [ ]:
df.loc[df.zscore<=-2, ('positions_GLD_Long', 'positions_GDX_Long')]=[1, -1] # Buy spread

In [ ]:
df.loc[df.zscore<=1, ('positions_GLD_Short', 'positions_GDX_Short')]=0 # Exit short spread

In [ ]:
df.loc[df.zscore>=-1, ('positions_GLD_Long', 'positions_GDX_Long')]=0 # Exit long spread

In [ ]:
df.fillna(method='ffill', inplace=True) # ensure existing positions are carried forward unless there is an exit signal

In [ ]:
positions_Long=df.loc[:, ('positions_GLD_Long', 'positions_GDX_Long')]

In [ ]:
positions_Short=df.loc[:, ('positions_GLD_Short', 'positions_GDX_Short')]

In [ ]:
positions=np.array(positions_Long)+np.array(positions_Short)

In [ ]:
positions=pd.DataFrame(positions)

In [ ]:
dailyret=df.loc[:, ('Adj Close_GLD', 'Adj Close_GDX')].pct_change()

In [ ]:
pnl=(np.array(positions.shift())*np.array(dailyret)).sum(axis=1)

In [ ]:
sharpeTrainset=np.sqrt(252)*np.mean(pnl[trainset[1:]])/np.std(pnl[trainset[1:]])

In [ ]:
sharpeTrainset

In [ ]:
sharpeTestset=np.sqrt(252)*np.mean(pnl[testset])/np.std(pnl[testset])

In [ ]:
sharpeTestset

In [ ]:
plt.plot(np.cumsum(pnl[testset]))

In [ ]:
positions.to_pickle('example3_6_positions')